## 研究能不能直接将源码中的HAN用openhgnn的HAN来实现

In [1]:
import os
import warnings

import torch

warnings.filterwarnings("ignore")

os.environ["DGLBACKEND"] = "pytorch"

import dgl
import torch.nn as nn
from openhgnn.dataset.NodeClassificationDataset import OHGB_NodeClassification
from openhgnn.utils import extract_metapaths, get_ntypes_from_canonical_etypes

In [2]:
acm = OHGB_NodeClassification(
    dataset_name="ohgbn-acm", raw_dir="./dataset", logger=None
)

Extracting file to ./openhgnn/dataset\ohgbn-acm
Done saving data into cached files.


In [5]:
# 数据集获得的hg和meta_paths_dict，即用户输入
hg = acm.g

In [7]:
ntypes = get_ntypes_from_canonical_etypes(hg.canonical_etypes)
ntype_meta_paths_dict = {}
for ntype in ntypes:
    ntype_meta_paths_dict[ntype] = {}
    for meta_path_name, meta_path in acm.meta_paths_dict.items():
        if meta_path[0][0] == ntype:
            ntype_meta_paths_dict[ntype][meta_path_name] = meta_path
print("原来就有的metapath")
print(ntype_meta_paths_dict)

print("\n加上extract的metapath")
for ntype, meta_paths_dict in ntype_meta_paths_dict.items():
    if len(meta_paths_dict) == 0:
        ntype_meta_paths_dict[ntype] = extract_metapaths(ntype, hg.canonical_etypes)
print(ntype_meta_paths_dict)

原来就有的metapath
{'paper': {'PAP': [('paper', 'paper-author', 'author'), ('author', 'author-paper', 'paper')], 'PSP': [('paper', 'paper-subject', 'subject'), ('subject', 'subject-paper', 'paper')]}, 'subject': {}, 'author': {}}

加上extract的metapath
{'paper': {'PAP': [('paper', 'paper-author', 'author'), ('author', 'author-paper', 'paper')], 'PSP': [('paper', 'paper-subject', 'subject'), ('subject', 'subject-paper', 'paper')]}, 'subject': {'mp0': [('subject', 'subject-paper', 'paper'), ('paper', 'paper-subject', 'subject')]}, 'author': {'mp0': [('author', 'author-paper', 'paper'), ('paper', 'paper-author', 'author')]}}


In [9]:
in_dim = 1902
hidden_dim = 512
out_dim = 256
dropout = 0.5
layer_num_heads = 1

In [11]:
import torch.nn.functional as F
from dgl.nn import GATConv
from openhgnn.layers import MetapathConv
from openhgnn.layers.macro_layer.SemanticConv import SemanticAttention
from openhgnn.models import HAN

openhgnn中HAN里的HANLayer
```python
class HANLayer:
    def __init__(self, meta_paths_dict, in_dim, out_dim, layer_num_heads, dropout):
        super(HANLayer, self).__init__()
        self.meta_paths_dict = meta_paths_dict
        semantic_attention = SemanticAttention(in_size=out_dim * layer_num_heads)
        mods = nn.ModuleDict({mp: GATConv(in_dim, out_dim, layer_num_heads,
                                          dropout, dropout, activation=F.elu,
                                          allow_zero_in_degree=True) for mp in meta_paths_dict})
        self.model = MetapathConv(meta_paths_dict, mods, semantic_attention)
        self._cached_graph = None
        self._cached_coalesced_graph = {}

    def forward(self, g, h):
        # For full batch, it is a heterogeneous graph.
        # For mini batch, it is a dict from mata path name to DGLBlock.

        # mini batch
        if isinstance(g, dict):
            h = self.model(g, h)
        
        # full batch
        else:
            if self._cached_graph is None or self._cached_graph is not g:
                self._cached_graph = g
                self._cached_coalesced_graph.clear()
                for mp, mp_value in self.meta_paths_dict.items():
                    self._cached_coalesced_graph[mp] = dgl.metapath_reachable_graph(
                        g, mp_value)
            h = self.model(self._cached_coalesced_graph, h)
        
        return h
```

`if self._cached_graph is None or self._cached_graph is not g:`，这一步判断了g是否是以前的g，说明可以根据这个来改变masked_gs






In [16]:
mods = nn.ModuleDict(
    {
        mp: GATConv(
            in_dim,
            out_dim,
            layer_num_heads,
            dropout,
            dropout,
            activation=F.elu,
            allow_zero_in_degree=True,
        )
        for mp in acm.meta_paths_dict
    }
)
semantic_attention = SemanticAttention(in_size=out_dim * layer_num_heads)

In [18]:
# HAN的forward中，要将hg根据mp切片为DGLBlock

_cached_coalesced_graph={}
for mp_name, mp in acm.meta_paths_dict.items():
    _cached_coalesced_graph[mp_name]=dgl.metapath_reachable_graph(hg,mp)

# 这一步将hg转换为了dict{metapath_name:mp-based graph(DGLBlock)}
print(_cached_coalesced_graph['PAP'])

g_dict=_cached_coalesced_graph

Graph(num_nodes=3025, num_edges=29436,
      ndata_schemes={'pspap_m2v_emb': Scheme(shape=(64,), dtype=torch.float32), 'psp_m2v_emb': Scheme(shape=(64,), dtype=torch.float32), 'pap_m2v_emb': Scheme(shape=(64,), dtype=torch.float32), 'train_mask': Scheme(shape=(), dtype=torch.uint8), 'test_mask': Scheme(shape=(), dtype=torch.uint8), 'label': Scheme(shape=(), dtype=torch.int64), 'h': Scheme(shape=(1902,), dtype=torch.float32), 'valid_mask': Scheme(shape=(), dtype=torch.uint8)}
      edata_schemes={})


HAN中使用到了MetapathConv
```python
class MetapathConv(nn.Module):
    def __init__(self, meta_paths_dict, mods, macro_func, **kargs):
        super(MetapathConv, self).__init__()
        # One GAT layer for each meta path based adjacency matrix
        self.mods = mods
        self.meta_paths_dict = meta_paths_dict
        self.SemanticConv = macro_func

    def forward(self, g_dict, h_dict):
        outputs = {g.dsttypes[0]: [] for s, g in g_dict.items()}

        for meta_path_name, meta_path in self.meta_paths_dict.items():
            new_g = g_dict[meta_path_name]

            # han minibatch
            if h_dict.get(meta_path_name) is not None:
                h = h_dict[meta_path_name][new_g.srctypes[0]]
            # full batch
            else:
                h = h_dict[new_g.srctypes[0]]
            outputs[new_g.dsttypes[0]].append(self.mods[meta_path_name](new_g, h).flatten(1))
        # semantic_embeddings = th.stack(semantic_embeddings, dim=1)  # (N, M, D * K)
        # Aggregate the results for each destination node type
        rsts = {}
        for ntype, ntype_outputs in outputs.items():
            if len(ntype_outputs) != 0:
                rsts[ntype] = self.SemanticConv(ntype_outputs)  # (N, D * K)
        return rsts
```

In [22]:
with torch.no_grad():
    outputs = {g.dsttypes[0]: [] for s, g in g_dict.items()}
    h_dict=hg.ndata["h"]
    for meta_path_name, meta_path in acm.meta_paths_dict.items():
        new_g = g_dict[meta_path_name]
    
        # han minibatch
        if h_dict.get(meta_path_name) is not None:
            h = h_dict[meta_path_name][new_g.srctypes[0]]
        # full batch
        else:
            h = h_dict[new_g.srctypes[0]]
        outputs[new_g.dsttypes[0]].append(mods[meta_path_name](new_g, h).flatten(1))
    print(outputs)
    print()
    # outputs是一个dict，属于paper的h，两条路径的聚合
    rsts = {}
    for ntype, ntype_outputs in outputs.items():
        if len(ntype_outputs) != 0:
            rsts[ntype] = semantic_attention(ntype_outputs)  # (N, D * K)
    
    print(rsts)

{'paper': [tensor([[ 0.2036, -0.0361,  0.1997,  ..., -0.3058,  0.1649, -0.1130],
        [ 0.3468, -0.3744, -0.1728,  ...,  0.1059, -0.3072, -0.2183],
        [ 0.2422,  0.0169,  0.1996,  ..., -0.1124, -0.0155,  0.1981],
        ...,
        [ 0.1022,  0.1159,  0.2687,  ..., -0.2894,  0.0432,  0.2256],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.7990,  0.2888,  0.8461,  ..., -0.8822,  0.4676, -0.3560]]), tensor([[ 0.6023, -0.0884,  0.8597,  ..., -0.5670,  0.2021, -0.0894],
        [ 0.0379,  0.0165,  0.2483,  ..., -0.3006,  0.2311, -0.2632],
        [ 0.0221,  0.0193,  0.1875,  ..., -0.2636,  0.1656, -0.2307],
        ...,
        [ 0.0624,  0.0572,  0.2304,  ..., -0.2181,  0.1100, -0.2251],
        [ 0.0486, -0.0011,  0.1911,  ..., -0.2555,  0.1922, -0.2325],
        [ 0.0262,  0.0360,  0.1845,  ..., -0.2480,  0.1602, -0.2313]])]}

{'paper': tensor([[ 3.9548e-01, -6.1275e-02,  5.1738e-01,  ..., -4.3152e-01,
          1.8284e-01, -1.0162e-01],
    

In [25]:
# 直接使用MetapathConv
mpconv = MetapathConv(acm.meta_paths_dict, mods, semantic_attention)
with torch.no_grad():
    out=mpconv(g_dict, hg.ndata["h"])
out

{'paper': tensor([[ 0.4138, -0.0761,  0.4194,  ..., -0.2695,  0.1206, -0.3403],
         [ 0.3455, -0.2130,  0.4541,  ..., -0.1536, -0.2452, -0.1454],
         [ 0.4710, -0.1164,  0.7756,  ..., -0.1699, -0.0819,  0.0316],
         ...,
         [ 0.0126,  0.0516,  0.1142,  ..., -0.1452,  0.0434, -0.0517],
         [ 0.1937, -0.0087,  0.3265,  ..., -0.2583, -0.0323, -0.0156],
         [ 0.0401,  0.0018,  0.0983,  ..., -0.1294,  0.0705, -0.1208]])}

In [27]:
out['paper'].shape

torch.Size([3025, 256])

## 直接使用HAN

In [35]:
# openhgnn的HAN需要ntype_meta_paths_dict
ntypes = get_ntypes_from_canonical_etypes(hg.canonical_etypes)



ntype_meta_paths_dict = {}
for ntype in ntypes:
    ntype_meta_paths_dict[ntype] = {}
    for meta_path_name, meta_path in acm.meta_paths_dict.items():
        if meta_path[0][0] == ntype:
            ntype_meta_paths_dict[ntype][meta_path_name] = meta_path
print(ntype_meta_paths_dict)

{'paper': {'PAP': [('paper', 'paper-author', 'author'), ('author', 'author-paper', 'paper')], 'PSP': [('paper', 'paper-subject', 'subject'), ('subject', 'subject-paper', 'paper')]}, 'subject': {}, 'author': {}}


In [37]:
han=HAN(ntype_meta_paths_dict,in_dim,hidden_dim,out_dim,[1],dropout=0.5)


In [39]:
with torch.no_grad():
    out=han.forward(hg,h_dict)
    print(out)
    print(out['paper'].shape)

{'paper': tensor([[ 3.1669e-02, -1.5434e-01,  6.7913e-02,  ..., -1.1997e-02,
         -8.9962e-02,  1.2028e-01],
        [ 1.0401e-01,  7.3429e-03,  1.1457e-02,  ..., -4.1890e-02,
         -1.1310e-01,  4.2793e-02],
        [ 1.5618e-01, -6.3225e-02, -9.1707e-02,  ...,  7.1752e-02,
         -1.5045e-01,  1.2942e-01],
        ...,
        [ 5.5060e-02, -4.7810e-02,  1.2178e-01,  ...,  3.6551e-03,
         -1.7644e-01, -2.6895e-02],
        [ 4.6538e-02, -5.7068e-02, -2.5474e-05,  ...,  5.6420e-02,
         -7.8086e-02,  3.3113e-02],
        [ 5.6040e-02, -5.0592e-02, -1.4845e-03,  ...,  5.4366e-02,
         -8.4831e-02,  3.6234e-02]])}
torch.Size([3025, 256])
